# Creating new Product Images with Compute
__________________

The Compute module provides scalable compute capabilities to parallelize your computations. Compute enables users to package and execute your Python code within nodes hosted on Descartes Lab's cloud infrastructure. These nodes offer the ability to access imagery at extremely high rates of throughput to execute computations over nearly any spatio-temporal scale. 

In this example notebook, we will create a [**Function**](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function) to calculate and create an NDVI product from Sentinel-2 L2A imagery then test it over Yakima County in Washington. The Yakima Valley contains ~75% of the total US hop acreage. 

First, we'll start by importing the descarteslabs Python client and the compute module.

In [ ]:
import descarteslabs as dl
from descarteslabs.catalog import Product, SpectralBand, properties as p

In [ ]:
from descarteslabs.compute import Function, Job

Now that we have the compute module, let's look at the primary objects we'll be working with: Function and Jobs.
 * **Function:** dynamically created, serverless functions containing user submitted, compiled code that you can submit many jobs to.
 * **Job:** submitted request for a single invocation of a created Function. 
 
As a hands-on example, we'll create a local method to search for Sentinel-2 imagery over a given AOI, calculate NDVI from the red and near-infrared bands, and upload the NDVI image to our catalog product using our Catalog module. Then, we'll create a **Function** containing the NDVI method. 

First, we'll create an empty `Product` to write our results in to, including a `Class Band`:

In [ ]:
# Add Unique ID to prevent conflicting products across your organization
from uuid import uuid4

# Get your org for namespace
org = dl.auth.Auth().payload["org"]
# Create NDVI Catalog product
product = Product.get_or_create(
    id=f"{org}:sentinel-2_ndvi-{uuid4()}",
    name="Sentinel-2 L2A NDVI",
)
product.save()

In [ ]:
# Create NDVI band for product
band = SpectralBand(
    product=product,
    name="ndvi",
    band_index=0,
    data_type="Float64",
    nodata=0,
    data_range=(0.0, 1.0),
    display_range=(0.0, 0.4),
)
band.save()
print(f"Saved {band}")

In [ ]:
print("Product status: ", product.state)  # check that product is 'saved'
print("Product ID: ", product.id)  # Get product ID that we will pass to function later

Next we'll define our Python function to wrap into a `Compute Function` below:

In [ ]:
# NDVI method
def create_ndvi_image(tile_key, start_date, end_date, product_id):
    import descarteslabs as dl

    # Import Catalog module methods
    from descarteslabs.catalog import Image, Product, properties as p

    # Get DLTile GeoContext for AOI
    dltile = dl.geo.DLTile.from_key(tile_key)

    # Find Sentinel-2 imagery over AOI
    print("Searching for imagery")
    images = (
        Product.get("esa:sentinel-2:l2a:v1")
        .images()
        .intersects(dltile)
        .filter(start_date < p.acquired <= end_date)
        .filter(p.cloud_fraction < 0.1)
    ).collect()
    print(f"Found {len(images)} images")

    # Create stack of red and nir bands for NDVI calc
    mosaic, raster_info = images.mosaic(["nir", "red"], raster_info=True)

    nir = mosaic[0]
    red = mosaic[1]

    # Calculate NDVI
    print("Calculating NDVI")
    ndvi = (nir - red) / (nir + red)

    # Create image for upload
    ndvi_image = Image(
        name=tile_key.replace(":", "_"),
        geometry=dltile.geometry,
        product_id=product_id,
        acquired=end_date,
    )

    # Upload image to catalog product
    upload = ndvi_image.upload_ndarray(ndvi, raster_meta=raster_info, overwrite=True)
    print("NDVI image upload ID:", upload.id)

    return upload.id

### Define AOI: Yakima County, Washington state
Now that we have our function, let's define our AOI to calculate NDVI over.

In [ ]:
import geopandas as gpd

# Get Yakima County Zip File
yak = gpd.read_file("../catalog/data/yakima.geojson")
# Collect bounds for Yakima county
minx, miny, maxx, maxy = yak.total_bounds

In [ ]:
from shapely.geometry import box

# Create bounding box to get DLTiles over
bbox = box(minx, miny, maxx, maxy)
# Create DLTile GeoContext objects to iterate over for NDVI function
dltiles = dl.geo.DLTile.from_shape(
    bbox, resolution=30, tilesize=256, pad=0  # 30 meters
)
# Get list of DLTile keys
dltile_keys = [tile.key for tile in dltiles]

print("Number of DLTiles: ", len(dltiles))
print("Single DLTile example: ")
print(dltiles[0])

### Testing function
Let's test the NDVI method locally and see the images uploaded to the Catalog product we created.

In [ ]:
# Test start and end dates
start_date = "2021-06-01"
end_date = "2021-06-15"
# Submit request for NDVI image upload
ndvi_test = create_ndvi_image(
    dltile_keys[0], product_id=product.id, start_date=start_date, end_date=end_date
)

In [ ]:
# Compare upload ID
product.image_uploads().collect()
# Check if new images are in Catalog product
img = product.images().filter(start_date < p.acquired <= end_date).collect()
img

In [ ]:
test_arr = img[0].ndarray("ndvi")
test_arr.shape

In [ ]:
# Plot example of image
dl.utils.display(test_arr, title="NDVI test", size=5, colormap="viridis")

### Create Compute function


In [ ]:
async_func = Function(
    create_ndvi_image,
    name="NDVI-from-sentinel",
    image="python3.9:latest",
    cpus=1,
    memory=2,
    timeout=60 * 10,  # in seconds ~ currently 10 min # Must be less than 900
    maximum_concurrency=25,
    retry_count=2,
)
async_func.save()

In [ ]:
async_func.id

### Submit jobs to Compute function

Now that we have a function built, we can test submitting Jobs. First, we'll submit a single job by creating a `compute.Job` object and wait for it to complete.

In [ ]:
async_func = Function.get(id=async_func.id)
async_func

In [ ]:
# Get the DLTile key for the first tile
key = dltiles[0].key
# Set a start_date and end_date
start_date = "2021-06-01"
end_date = "2021-06-15"

In [ ]:
# Create the job function
job = Job(async_func.id, args=[key, start_date, end_date, product.id])
job.save()

In [ ]:
%timeit job.wait_for_completion()

In [ ]:
# Look at the results of this product
job.log()

#### Plot example image

In [ ]:
# Check if new images are in Catalog product
job_image = product.images().filter(start_date < p.acquired <= end_date).collect()
job_image

job_arr = job_image[0].ndarray("ndvi")
job_arr.shape

# Plot example of image
dl.utils.display(job_arr, title="Job array", size=5, colormap="viridis")

### Submit multiple jobs
We can also submit multiple jobs to the same function.

### Submit many jobs using `Function.map()`

In [ ]:
func = Function.get(id=async_func.id)
func

##### Create dictionary of arguments to pass as Jobs


In [ ]:
# Convenience function for building kwarg dict
def get_bulk_kwargs(jobs, **kwargs):
    bulk_kwargs = []
    for _ in range(jobs):
        bulk_kwargs.append(dict(kwargs))

    return bulk_kwargs

In [ ]:
## Collect args to submit to Function
# Get a list of the DLTile keys
dltile_keys = [
    [tile.key] for tile in dltiles
]  # First iterable argument needs to be list of lists - List[List]
# Test start and end dates
start_date = "2021-06-01"
end_date = "2021-06-15"

# Build kwarg dict
kwargs = get_bulk_kwargs(
    len(dltiles), start_date=start_date, end_date=end_date, product_id=product.id
)
print("Example of Key word args")
kwargs[0:5]

In [ ]:
# Submit multiple Jobs using map
jobs = func.map(dltile_keys, kwargs)

Once these jobs have completed, results can be received by iterating through the jobs associated with our Function object. 

In [ ]:
# Get results
statuses = {}

for j in async_func.jobs:
    stat = j.status
    if stat in statuses:
        job_info = {"id": j.id, "result": j.result}
        statuses[stat].append(j.id)
    else:
        statuses[stat] = [{"id": j.id, "result": j.result}]

### Submit Multiple jobs by creating multiple Job objects

Jobs can also be create directly and will be executed once the object is saved. I.e., `job.save()`

In [ ]:
jobs = []
for key in dltile_keys:
    j = Job(async_func.id, args=[key, "2022-06-01", "2022-06-30", product.id])
    j.save()
    jobs.append(j)

In [ ]:
async_func.wait_for_completion()

## Finding previous `Functions`

Until the Compute Monitor is finalized, we will need to search for previous functions programmatically.

In [ ]:
# Get active functions by specifying the "running" status
active_funcs = [{"name": f.name, "id": f.id} for f in Function.list(status="running")]
active_funcs

In [ ]:
# Filter Functions by name prefixes
funcs = Function.list()
ndvi_funcs = [f.name for f in funcs if f.name.startswith("NDVI")]
ndvi_funcs